# 2.4 Implementing proofs using the order $(\mathbb{N}^n, <_\text{lex})$

### Using Z3 online

In [40]:
def clump_left(string, index):
    if string[index] in [')','}']:
        level = 1
        i = index - 1
        while level != 0:
            if string[i] in [')','}']:
                level = level + 1
            elif string[i] in ['(','{']:
                level = level - 1
            i = i - 1
        return i + 1
    else:
        i = index
        while i > 0 and (string[i-1].isnumeric() or string[i-1]=='x'):
            i = i - 1
        return i
    
def clump_right(string, index):
    if string[index] in ['(','{']:
        level = 1
        i = index + 1
        while level != 0:
            if string[i] in ['(','{']:
                level = level + 1
            elif string[i] in [')','}']:
                level = level - 1
            i = i + 1
        return i - 1
    else:
        i = index
        while i < len(string)-1 and (string[i+1].isnumeric() or string[i+1]=='x'):
            i = i + 1
        return i
    
def insert(string, expression, index):
    return string[:index] + expression + string[index:]

def operator_process(string, operator_list):
    i = 0
    while i < len(string):
        if string[i] in operator_list:
            l = clump_left(string, i-1)
            r = clump_right(string, i+1)
            string = insert(string, '(', l)
            string = insert(string, ')', r+2)
            i = i + 2
        i = i + 1
    return string

def place_parenthesis(string):
    #given an expression in the usual way the program returns
    #the expression with parenthesis
    
    operator_priority = [['^'],['*','/'],['+','-']]
    string = string.replace('(', '{')
    string = string.replace(')', '}')
    for operators in operator_priority:
        string = operator_process(string, operators)
    string = string.replace('{', '')
    string = string.replace('}', '')
    return string

#Ex: place_parenthesis('(1+3)^2*6-5/(2)', [['^'],['*','/'],['+','-']])

In [41]:
def converter(string:str):
    #From an expression with parenthesis (like the output of place_parenthesis) 
    #it returns expressions of the form needed to Z3
    
    operation_list = ['^','*','/','+','-']
    
    def operations_counter(string):
        counter = 0
        for c in string:
            if c in operation_list:
                counter += 1
        return counter
    
    def aux(string):
        if operations_counter(string)==0: #f(x) onde f(x) não tem operações -> f(x)
            return string
        #(x1+x2) -> (+ x1 x2)
        else: #(f(x) + g(x)) -> (+ aux(f(x)) aux(g(x)))
            string = string[1:-1]
            if '(' not in string:
                i = 0
                found = False
                while i<len(string) and not found:
                    if string[i] in operation_list:
                        found = True
                    else:
                        i+=1
                        
            else:
                level = 0
                i = 0
                if string[i]=='(':
                    level -= 1
                elif string[i]==')':
                    level += 1
                while i<len(string) and level!=0:
                    i += 1
                    if string[i]=='(':
                        level -= 1
                    elif string[i]==')':
                        level += 1
                i += 1                                      
                
            op = string[i]
            s1 = string[:i]
            s2 = string[i+1:]

            return '(%s %s %s)'%(op, aux(s1), aux(s2))

    return aux(string)

In [42]:
def to_check(n,f,G):
    #It returns the commands to run on https://microsoft.github.io/z3guide/playground/Freeform%20Editing/ 
    #to test if not(f(g(x))<f(x)) is unsat
    
    #G é da forma (..., ..., ... ...)
    
    print('\nRun the following:')
    print('-----')
    #declaração das constantes
    print(';declare constant')
    for i in range(n):
        print('(declare-const x%d Int)'%(i+1))
    
    for i in range(n):
        print('(assert (> x%d 0))'%(i+1))
        
    #retirar espaços a G
    g = ''
    for c in G:
        if c!=' ':
            g += c
    g = g[1:-1] #retirar os parêntesis
    
    comma_index = [i for i in range(len(g)) if g[i]==',']
    gs = [g[0:comma_index[0]]] #projeção 1 de g
    gs += [g[comma_index[i]+1:comma_index[i+1]] for i in range(len(comma_index)-1)]
    gs += [g[comma_index[-1]+1:]] #projeção m de g
        
    fog = f #substituir as ocorrências em f de x1 por g1(x), x2 por g2(x), etc.
    for i in range(n):
        fog = fog.replace('x%d'%(i+1), gs[i])
    
    print('(assert (not (< %s %s)))'%(converter(place_parenthesis(fog)), converter(place_parenthesis(f))))
    print('(echo "Answer:") ')
    print('(check-sat)')
    print('-----')

In [43]:
def number_of_variables(exp:str):
    #It returns the number of variables of the input expression 
    list_variable = []
    i = 0
    while i<len(exp):
        if exp[i]=='x':
            i += 1
            variable = ''
            while i<len(exp) and exp[i] in ['0','1','2','3','4','5','6','7','8','9']:
                variable += exp[i]
                i += 1
            list_variable += [int(variable)]
        else:
            i+=1
            
    return max(list_variable)

In [44]:
def main():
    #o output é 1 se o programa termina, 0 se não termina e 'unknown' caso não saiba.
    m = int(input('m = '))
    f = input('f(x) = ')
    ans = 'unsat'
    n_f = number_of_variables(f)
    
    i = 1
    while i<=m and ans=='unsat':
        g = input('g%d = '%i)
        n_g = number_of_variables(g)
        to_check(max(n_f,n_g),f,g)
        ans = input('Answer: ')
        i += 1
    
    if ans == 'unsat':
        print('\nThe program is finite.')
        return 1
    
    elif ans == 'sat':
        print('\nThe programa is not finite.')
        return 0
    
    else: #ans == 'unknown'
        print('\nunknown')
        return 'unknown' 

### Example - Program 4

    (x, y, z) = (input(Z), input(Z), input(Z))
    while x>0 and y>0 and z>0
        control = input(1, 2, 3)
        if control == 1
            (x, y, z) = (x+1, y−1, z−1)
        else if control == 2
            (x, y, z) = (x−1, y+1, z−1)
        else if control == 3
            (x, y, z) = (x−1, y−1, z+1)

In [45]:
main()

m = 3
f(x) = x1+x2+x3
g1 = (x1+1, x2-1, x3-1)

Run the following:
-----
;declare constant
(declare-const x1 Int)
(declare-const x2 Int)
(declare-const x3 Int)
(assert (> x1 0))
(assert (> x2 0))
(assert (> x3 0))
(assert (not (< (- (+ (- (+ (+ x1 1) x2) 1) x3) 1) (+ (+ x1 x2) x3))))
(echo "Answer:") 
(check-sat)
-----
Answer: unsat
g2 = (x1-1, x2+1, x3-1)

Run the following:
-----
;declare constant
(declare-const x1 Int)
(declare-const x2 Int)
(declare-const x3 Int)
(assert (> x1 0))
(assert (> x2 0))
(assert (> x3 0))
(assert (not (< (- (+ (+ (+ (- x1 1) x2) 1) x3) 1) (+ (+ x1 x2) x3))))
(echo "Answer:") 
(check-sat)
-----
Answer: unsat
g3 = (x1-1, x2-1, x3+1)

Run the following:
-----
;declare constant
(declare-const x1 Int)
(declare-const x2 Int)
(declare-const x3 Int)
(assert (> x1 0))
(assert (> x2 0))
(assert (> x3 0))
(assert (not (< (+ (+ (- (+ (- x1 1) x2) 1) x3) 1) (+ (+ x1 x2) x3))))
(echo "Answer:") 
(check-sat)
-----
Answer: unsat

The program is finite.


1

### Using Z3 on Python

In [23]:
from z3 import *

In [24]:
def function_trusted(f:str):
    #returns 1 iff all characters used are admissible
    char_trusted = ['(',')','^','*','/','+','-','0','1','2','3','4','5','6','7','8','9','x','n','m']
    i = 0
    ok = True
    while i<len(f) and ok:
        if f[i] not in char_trusted:
            ok = False
        else:
            i += 1
    return ok

In [25]:
def number_of_variables(exp:str, subexp:str):
    #It returns the number of variables of the input expression 
    list_variable = [0]
    len_exp    = len(exp)
    len_subexp = len(subexp)
    i = 0
    while i < len_exp-len_subexp:
        if exp[i:i+len_subexp]==subexp:
            i += len_subexp
            variable = '0'
            while i<len_exp and exp[i] in ['0','1','2','3','4','5','6','7','8','9']:
                variable += exp[i]
                i += 1
            list_variable += [int(variable)]
        else:
            i+=1
            
    return max(list_variable)

In [26]:
def solver_N_lex(n_variables:int, F:str, G:str) -> str:
    #It returns the SMT value of not(g(x)<x) (unsat, sat or unkown)    
    #G is of the form (..., ..., ... ..., ...)
    
    s = Solver()
    
    #declaração das constantes
    n = Int('n')
    nn = Int('nn')
    s.add(nn>0)
    for i in range(n_variables):
        exec("x%d = Int('x%d')"%(i+1,i+1))
        exec("s.add(x%d > 0)"%(i+1))
        
    #retirar espaços a G
    g = ''
    for c in G:
        if c!=' ':
            g += c
    if(g[0]=='('):
        g = g[1:-1] #retirar os parêntesis
    
    comma_index = [i for i in range(len(g)) if g[i]==',']
    if comma_index != 0:
        gs = [g[0:comma_index[0]]] #projeção 1 de g
        gs += [g[comma_index[i]+1:comma_index[i+1]] for i in range(len(comma_index)-1)]
        gs += [g[comma_index[-1]+1:]] #projeção m de g
    else:
        gs = [g]
    
    for function_g in gs:
        assert function_trusted(function_g)
    
    #retirar espaços a f
    f = ''
    for c in F:
        if c!=' ':
            f += c
    if f[0] == '(':
            f = f[1:-1] #retirar os parêntesis
            
    comma_index = [i for i in range(len(f)) if f[i]==',']
    if len(comma_index)!=0:
        fs = [f[0:comma_index[0]]] #projeção 1 de f
        fs += [f[comma_index[i]+1:comma_index[i+1]] for i in range(len(comma_index)-1)]
        fs += [f[comma_index[-1]+1:]] #projeção m de f
    else:
        fs = [f]
    
    for function_f in fs:
        assert function_trusted(function_f)
        
    fog = f.upper() #substituir as ocorrências em f de x1 por g1(x), x2 por g2(x), etc.
    for i in range(n_variables):
        fog = fog.replace('X%d'%(i+1), gs[i])
    
    comma_index = [i for i in range(len(fog)) if fog[i]==',']
    if len(comma_index)!=0:
        fgs = [fog[0:comma_index[0]]] #projeção 1 de fog
        fgs += [fog[comma_index[i]+1:comma_index[i+1]] for i in range(len(comma_index)-1)]
        fgs += [fog[comma_index[-1]+1:]] #projeção m de fog
    else:
        fgs = [fog]
    

    formula = 'Or(' # ((fog)1<f1) or ((fog)1==f1 and (fog)2<f2) or ... or ((fog)1==f1 and (fog)2==f2 and ... and (fog)(n-1)==f(n-1) and (fog)n<fn)
    for i in range(0, len(comma_index)+1):
        if i==0:
            aux = '(eval(fgs[0])<eval(fs[0]))'
        else:
            aux = 'And(' #for example (x1=g1 and x2=g2 and x3<g3)
            for j in range(0, i):
                if j<i-1:
                    aux += 'eval(fgs[%d])==eval(fs[%d]), '%(j,j)
                else:
                    aux += 'eval(fgs[%d])==eval(fs[%d])'%(j,j)
            aux += ', eval(fgs[%d])<eval(fs[%d]))'%(i,i)
        formula += aux + ', '
    formula = formula[:-2] + ')'
    
    formula = eval(formula)
    
    s.add(Not(formula))

    return str(s.check())

In [27]:
def main():
    #o output é 1 se o programa termina, 0 se não termina e 'unknown' caso não saiba.
    m = int(input('m = '))
    f = input('f(x) = ')
    
    print('\nx = (input(Z), ..., input(Z))')
    print('while x > 0:')
    control = ''
    if m<6:
        for i in range(m):
            control += '%d,'%(i+1)
        control = control[:-1] #to exclude the last comma
    else:
        control = '1, 2, ..., %d, %d'%(m-1,m)

    print('   command = input(%s)'%control)
    
    i = 1
    ans = 'unsat'
    n_f = number_of_variables(f, 'x')
    while i<=m and ans=='unsat':
        print('   if command == %d:'%i)
        g = input('      x = ')
        n_g = number_of_variables(g, 'x')

        ans = solver_N_lex(max(n_f, n_g), f, g)
        
        i += 1
    
    if ans == 'unsat':
        print('\nThe program is finite.')
        return 1
    
    elif ans == 'sat':
        print('\n\nThe programa is not finite.')
        return 0
    
    else: #ans == 'unknown'
        print('unknown')
        return 'unknown'

#### Example 1 - Program 4

In [29]:
main()

m = 3
f(x) = x1+x2+x3

x = (input(Z), ..., input(Z))
while x > 0:
   command = input(1,2,3)
   if command == 1:
      x = (x1-1,x2,x3)
   if command == 2:
      x = (x1-1,x2-1,x3+1)
   if command == 3:
      x = (x2,x1-1,x3)

The program is finite.


1

#### Example 2 - Program 5

In [39]:
main()

m = 3

x = (input(Z), ..., input(Z))
while x > 0:
   command = input(1,2,3)
   if command == 1:
      x = (x1-1, x2+nn, x3, x4)
   if command == 2:
      x = (x1, x2-1, x3+nn, x4)
   if command == 3:
      x = (x1, x2, x3-1, x4+nn)

The program is finite.


1

### 2.4.1 A proof that a syntactic unification algorithm terminates

In [35]:
def solver_N_lex(n_x:int, n_n:int, n_nn:int, G:str) -> str:
    #It returns the SMT value of not(g(x)<x) (unsat, sat or unkown)    
    #G is of the form (..., ..., ... ..., ...)
    
    s = Solver()
    
    #declaração das constantes
    nn = Int('nn')
    s.add(nn>0)
    for i in range(n_n):
        exec("n%d = Int('n%d')"%(i+1,i+1))
    for i in range(n_nn):
        exec("nn%d = Int('nn%d')"%(i+1,i+1))
        exec("s.add(Or(nn%d > 0, nn%d == 0))"%(i+1, i+1))        
    for i in range(n_x):
        exec("x%d = Int('x%d')"%(i+1,i+1))
        exec("s.add(x%d > 0)"%(i+1))
        
    #retirar espaços a G
    g = ''
    for c in G:
        if c!=' ':
            g += c
    g = g[1:-1] #retirar os parêntesis
    
    comma_index = [i for i in range(len(g)) if g[i]==',']
    gs = [g[0:comma_index[0]]] #projeção 1 de g
    gs += [g[comma_index[i]+1:comma_index[i+1]] for i in range(len(comma_index)-1)]
    gs += [g[comma_index[-1]+1:]] #projeção m de g
    
    for function_g in gs:
        assert function_trusted(function_g)
 
    formula = 'Or(' # (g1<x1) or (g1==x1 and g2<x2) or ... or (g1==x1 and g2==x2 and ... and g(n-1)==x(n-1) and gn<xn)
    for i in range(1, n_x+1):
        if i==1:
            aux = '(eval(gs[0])<x1)'
        else:
            aux = 'And(' #for example (x1=g1 and x2=g2 and x3<g3)
            for j in range(1, i):
                if j<i-1:
                    aux += 'eval(gs[%d])==x%d, '%(j-1,j)
                else:
                    aux += 'eval(gs[%d])==x%d'%(j-1,j)
            aux += ', eval(gs[%d])<x%d)'%(i-1,i)
        formula += aux + ', '
    formula = formula[:-2] + ')'
    
    formula = eval(formula)
    
    s.add(Not(formula))

    return str(s.check())

In [37]:
def main():
    #o output é 1 se o programa termina, 0 se não termina e 'unknown' caso não saiba.
    m = int(input('m = '))
    
    print('\nx = (input(Z), ..., input(Z))')
    print('while x > 0:')
    command = ''
    if m<6:
        for i in range(m):
            command += '%d,'%(i+1)
        command = command[:-1] #to exclude the last comma
    else:
        command = '1, 2, ..., %d, %d'%(m-1,m)

    print('   command = input(%s)'%command)
    
    i = 1
    ans = 'unsat'
    while i<=m and ans=='unsat':
        print('   if command == %d:'%i)
        g = input('      x = ')
        n_x_g  = number_of_variables(g, 'x')
        n_nn_g = number_of_variables(g, 'nn')
        n_n    = number_of_variables(g, 'n')

        ans = solver_N_lex(n_x_g, n_n, n_nn_g, g)
        
        i += 1
    
    if ans == 'unsat':
        print('\nThe program is finite.')
        return 1
    
    elif ans == 'sat':
        print('\n\nThe programa is not finite.')
        return 0
    
    else: #ans == 'unknown'
        print('unknown')
        return 'unknown'   

In [38]:
main()

m = 4

x = (input(Z), ..., input(Z))
while x > 0:
   command = input(1,2,3,4)
   if command == 1:
      x = (x1-nn1, x2, x3-nn)
   if command == 2:
      x = (x1-nn1, x2, x3-nn)
   if command == 3:
      x = (x1-nn2, x2-2, x3+nn1)
   if command == 4:
      x = (x1-nn, x2+nn1, x3+n1)

The program is finite.


1